In [2429]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")

In [2430]:
elo_data = pd.read_sql_query(
    "Select DISTINCT * From Elo_AllMatches",
    con=devengine,)
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [2431]:
elo_data['Elo_Diff']=elo_data['Elo_Fav_Elo']-elo_data['Elo_Dog_Elo']
elo_data['Elo_Odds_Diff']=elo_data['Elo_Fav_Odds']-elo_data['Elo_Dog_Odds']
elo_data=elo_data[((elo_data['Elo_Diff']).lt(150))!=True]
elo_data['Winner']=elo_data.apply(lambda x: 'Elo_Fav' if x['Elo_Fav']==x['Winner'] else 'Elo_Dog',axis=1)
elo_data=elo_data[(elo_data['Elo_Fav_Odds']>elo_data['Elo_Fav_Est_Odds'])&(elo_data['Elo_Fav_Odds']>1.79)]
#elo_data['Winner_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Odds'],axis=1)
#elo_data['Winner_Est_Odds']=elo_data.apply(lambda x: x['Elo_Fav_Est_Odds'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Est_Odds'],axis=1)
#elo_data['Winner_Elo']=elo_data.apply(lambda x: x['Elo_Fav_Elo'] if x['Elo_Fav']==x['Winner'] else x['Elo_Dog_Elo'],axis=1)
elo_data['Fav_Odds']=elo_data[['Elo_Fav_Odds','Elo_Dog_Odds']].min(axis=1)
elo_data['Dog_Odds']=elo_data[['Elo_Fav_Odds','Elo_Dog_Odds']].max(axis=1)
elo_data['Odds_Diff']=elo_data['Fav_Odds']-elo_data['Dog_Odds']
elo_data['Fav_Odds_Diff']=elo_data['Elo_Fav_Odds']-elo_data['Elo_Fav_Est_Odds']
elo_data['Dog_Odds_Diff']=elo_data['Elo_Dog_Odds']-elo_data['Elo_Dog_Est_Odds']
#elo_data['Total_Diff']=elo_data['Elo_Fav_Total']-elo_data['Elo_Dog_Total']
#elo_data=elo_data[elo_data['Elo_Fav_Odds']<1.8]
train_data=elo_data[elo_data['Date']<'2022-09-01']
test_data=elo_data[elo_data['Date']>='2022-09-01']
test_data_resulted=test_data.reset_index()
test_data=test_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Elo_Dog_Odds','Resulted','Time'])
train_data=train_data.drop(columns=['Surface','Date','Sex','Elo_Fav','Elo_Dog','Elo_Dog_Odds','Resulted','Time'])


In [2432]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from matplotlib import pyplot

In [2433]:
X = train_data.drop(["Winner"], axis=1)
y = train_data["Winner"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
model2 = LogisticRegression(max_iter=20000000)
#model2 = SVC()
model2.fit(X_train, y_train)
train_score2 = model2.score(X_train, y_train)
test_score2 = model2.score(X_test, y_test)
print(train_score2,test_score2)

0.608 0.5351351351351351


In [2434]:
importance=model2.coef_[0]
coefficients = pd.DataFrame({"Feature":X.columns,"Coefficients":np.transpose(model2.coef_[0], )})

print(coefficients)


             Feature  Coefficients
0       Elo_Fav_Odds      0.029908
1   Elo_Fav_Est_Odds     -0.023511
2   Elo_Dog_Est_Odds     -0.311846
3        Elo_Fav_Elo      0.004538
4        Elo_Dog_Elo     -0.004943
5      Elo_Fav_Total      0.001893
6      Elo_Dog_Total      0.001272
7           Elo_Diff      0.009481
8      Elo_Odds_Diff     -0.170878
9           Fav_Odds      0.226887
10          Dog_Odds      0.003807
11         Odds_Diff      0.223080
12     Fav_Odds_Diff      0.053419
13     Dog_Odds_Diff      0.512632


In [2435]:
pred2=model2.predict(test_data.drop(columns='Winner',axis=1))

In [2436]:
len(test_data_resulted)

60

In [2437]:
cols = [
    "Date",
    "Prediction",
    "Winner",
    "Elo_Fav",
    "Elo_Fav_Odds",
    "Elo_Dog",
    "Elo_Dog_Odds","Time","Resulted"
    # "Training_Accuracy",
    # "Testing_Accuracy",
]
df = pd.DataFrame(columns=cols)
List = []

In [2438]:
for index, tuples in test_data_resulted.iterrows():
        #if index <> len(test_data_resulted):

            values = [
                test_data_resulted["Date"][index],
                pred2[index],
                test_data_resulted["Winner"][index],
                test_data_resulted["Elo_Fav"][index],
                test_data_resulted["Elo_Fav_Odds"][index],
                test_data_resulted["Elo_Dog"][index],
                test_data_resulted["Elo_Dog_Odds"][index],
                test_data_resulted["Time"][index],
                test_data_resulted["Resulted"][index],
                #  "{:.0%}".format(train_score2),
                #  "{:.0%}".format(test_score2),
            ]
            zipped = zip(cols, values)
            a_dictionary = dict(zipped)
            List.append(a_dictionary)

In [2439]:
temp = pd.DataFrame(List)
df = pd.concat([df, temp])

In [2440]:
exclude_today=df[df['Date']!=time_now_formatted].copy()

In [2441]:
exclude_today['Predicton_Odds']=exclude_today.apply(lambda x: x['Elo_Fav_Odds'] if 'Elo_Fav' == x['Prediction'] else x['Elo_Dog_Odds'],axis=1)
#exclude_today['Winner_Odds']=exclude_today['Winner_Odds'].astype(float)
exclude_today_odds=exclude_today[((exclude_today['Predicton_Odds']).ge(1.9))&((exclude_today['Predicton_Odds']).le(2.3))].copy()
#exclude_today

In [2442]:
STAKE=50
exclude_today_odds['Profit']=exclude_today_odds.apply(lambda x: -STAKE if x['Winner']!=x['Prediction'] else (STAKE * x['Predicton_Odds'])-STAKE,axis=1)
exclude_today_odds['Profit'].sum()

In [2443]:
exclude_today_odds['Profit'].sum()

-98.5

In [2444]:
len(exclude_today_odds)

12

In [2445]:
len(exclude_today_odds[exclude_today_odds['Winner']==exclude_today_odds['Prediction']])/len(exclude_today_odds)

0.4166666666666667

In [2446]:
today = df[df["Date"] == time_now_formatted].copy()
today["Prediction_Odds"] = today.apply(
    lambda x: x["Elo_Fav_Odds"] if "Elo_Fav" == x["Prediction"] else x["Elo_Dog_Odds"],
    axis=1,
)
today = today[((today["Prediction_Odds"]).ge(1.8)) & ((today["Prediction_Odds"]).le(2.3))]


In [2447]:
today['Predicted_Player']=today.apply(lambda x: x['Elo_Fav'] if x['Prediction']=='Elo_Fav' else x['Elo_Dog'], axis=1)
today[['Time','Predicted_Player','Prediction_Odds']][today['Resulted']!='True'].sort_values(by='Time',ascending=True)

,Time,Predicted_Player,Prediction_Odds
59,01:30,Diego Schwartzman,1.9


In [2448]:
today_resulted=today[(today['Resulted']=='True')&((today['Prediction_Odds']).ge(1.8))&((today['Prediction_Odds']).le(2.5))].sort_values(by='Time',ascending=True)
len(today_resulted[today_resulted['Winner']==today_resulted['Prediction']])/len(today_resulted)


ZeroDivisionError: division by zero

In [ ]:
today_resulted

,Date,Prediction,Winner,Elo_Fav,Elo_Fav_Odds,Elo_Dog,Elo_Dog_Odds,Time,Resulted,Prediction_Odds,Predicted_Player
975,2022-10-21,Elo_Dog,Elo_Dog,Diego Schwartzman,1.90,David Goffin,1.89,01:30,False,1.89,David Goffin
978,2022-10-21,Elo_Dog,Elo_Dog,Tommy Paul,1.83,Mikael Ymer,1.96,02:00,False,1.96,Mikael Ymer
960,2022-10-21,Elo_Fav,Elo_Dog,Leolia Jeanjean,1.99,Kristina Mladenovic,1.79,03:00,False,1.99,Leolia Jeanjean
962,2022-10-21,Elo_Fav,Elo_Dog,Caroline Garcia,1.87,Sloane Stephens,1.93,08:00,False,1.87,Caroline Garcia
